# Cantilever Beam

In this tutorial, we'll create a cantilever beam simulation using SOFA physics. The simulation will show how the beam bends under gravity.

In [ ]:
import Sofa
import SofaRuntime
SofaRuntime.init()

First, we set up the basic simulation environment and define gravity and time step.

In [ ]:
root = Sofa.Core.Node("root")
root.gravity.value = [0, -9.81, 0]
root.dt.value = 0.001

Next, we configure the physics solvers and simulation loop.

In [ ]:
root.addObject("DefaultAnimationLoop")

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.ODESolver.Backward")
root.addObject("NewtonRaphsonSolver", maxNbIterationsNewton=1000, maxNbIterationsLineSearch=10)

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.ODESolver.Backward")
solver = root.addObject("StaticSolver")

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.LinearSolver.Iterative")
linear_solver = root.addObject("CGLinearSolver", iterations=25, tolerance=1e-5, threshold=1e-5)

Now we define the 3D grid structure and position storage.

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Topology.Container.Grid")
grid = root.addObject('RegularGridTopology', name="grid", nx=4, ny=4, nz=20, xmin=-9, xmax=-6, ymin=0, ymax=3, zmin=0, zmax=19)

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.StateContainer")
state = root.addObject("MechanicalObject", template="Vec3", name="state")

Add mass:

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Mass")
root.addObject("MeshMatrixMass", template="Vec3,Vec3", name="mass", totalMass=320)

Fixed constraint:

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Engine.Select")
box = root.addObject('BoxROI', name="box", box=[-10, -1, -0.0001,  -5, 4, 0.0001])

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.Constraint.Projective")
constraint = root.addObject('FixedProjectiveConstraint', indices="@box.indices")

Linear elasticity:

In [ ]:
SofaRuntime.importPlugin("Sofa.Component.SolidMechanics.FEM.Elastic")
fem = root.addObject('HexahedronFEMForceField', name="FEM", youngModulus=20000, poissonRatio=0.3, method="large")

Initialize the simulation:

In [ ]:
Sofa.Simulation.initRoot(root)

Perform one simulation step:

In [ ]:
Sofa.Simulation.animate(root, root.dt.value)

Visualize the result:

In [ ]:
import k3d
plot = k3d.plot()
plot += k3d.mesh(vertices=state.position.value, indices=grid.triangles.value,
                 opacity=0.25, color=0xE84E1B, name="Cantilever beam")
plot.display()